In [12]:
import os

In [13]:
%pwd

'd:\\Angus_Issues\\NorthCarolina_CameroonChapter_AngusIssues'

In [4]:
os.chdir("../")

In [14]:
from dataclasses import dataclass
from pathlib import Path
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import joblib

In [15]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_name: str
    order: tuple
    target_column: float

In [16]:
# Assuming the presence of helper functions and constants
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH 
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ETSModel
        schema = self.schema
        target_column = list(self.schema['TARGET_COLUMN'].keys())[0]

        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            data_path=config.data_path,
            model_name=config.model_name,
            order=(params.error, params.trend, params.seasonal),
            target_column=target_column
        )

        return model_trainer_config

In [18]:
import os
from mlProject import logger
import pandas as pd
from statsmodels.tsa.exponential_smoothing.ets import ETSModel

In [21]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):

        data = pd.read_csv(self.config.data_path)
        data.drop('Unnamed: 0', axis=1, inplace=True)
        data['Date'] = pd.to_datetime(data['Date']) # Convert 'Date' column to datetime format
        data = data.set_index('Date')

        # Initialize forecast results DataFrame
        forecast_results = pd.DataFrame(columns=['Date', 'Cow', 'Predicted Body Weight (kg)'])

        # Define the date to split the data (adjust the date as needed)
        split_date = pd.to_datetime('2022-02-15')

        # Extract unique cow IDs from the DataFrame
        unique_cows = data['Cow'].unique()
        

        # Continue with the forecasting logic
        for cow in unique_cows:
            cow_data = data[data['Cow'] == cow]

            # Ensure that the index is a DatetimeIndex
            cow_data.index = pd.to_datetime(cow_data.index)

            # Split the data into train and test based on the split_date
            cow_train_data = cow_data[cow_data.index < split_date]
            cow_test_data = cow_data[cow_data.index >= split_date]

            # Fit the model and make predictions
            model_ETS = ETSModel(cow_train_data['Body Weight (kg)'], error='add', trend=None, seasonal=None)
            model_fit = model_ETS.fit()

            # Make predictions for the test set
            cow_predictions = model_fit.predict(start=len(cow_train_data), end=len(cow_train_data) + len(cow_test_data) - 1)

            # Store the predictions
            cow_forecast_df = pd.DataFrame({
                'Date': cow_test_data.index,
                'Cow': cow,
                'Predicted Body Weight (kg)': cow_predictions
            })

            forecast_results = pd.concat([forecast_results, cow_forecast_df])

        forecast_results.reset_index(drop=True, inplace=True)
        print(forecast_results)


        # Ensure the directory exists
        model_dir = os.path.join(self.config.root_dir, 'model_trainer')
        os.makedirs(model_dir, exist_ok=True)

        forecast_results.to_csv(os.path.join(self.config.root_dir, 'forecast_results.csv'), index=False)


        joblib.dump(model_fit, os.path.join(self.config.root_dir, self.config.model_name))

In [22]:
# Main training execution
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-08-17 21:25:52,511: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-17 21:25:52,516: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-17 21:25:52,519: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-08-17 21:25:52,520: INFO: common: created directory at: artifacts]


d:\Angus_Issues\NorthCarolina_CameroonChapter_AngusIssues\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
d:\Angus_Issues\NorthCarolina_CameroonChapter_AngusIssues\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
d:\Angus_Issues\NorthCarolina_CameroonChapter_AngusIssues\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
d:\Angus_Issues\NorthCarolina_CameroonChapter_AngusIssues\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
d:\Angus_Issues\NorthCarolin

          Date     Cow  Predicted Body Weight (kg)
0   2022-02-15   Cow 1                  430.036713
1   2022-02-16   Cow 1                  430.036713
2   2022-02-17   Cow 1                  430.036713
3   2022-02-18   Cow 1                  430.036713
4   2022-02-19   Cow 1                  430.036713
..         ...     ...                         ...
895 2022-02-25  Cow 60                  456.172075
896 2022-02-26  Cow 60                  456.172075
897 2022-02-27  Cow 60                  456.172075
898 2022-02-28  Cow 60                  456.172075
899 2022-03-01  Cow 60                  456.172075

[900 rows x 3 columns]


d:\Angus_Issues\NorthCarolina_CameroonChapter_AngusIssues\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
d:\Angus_Issues\NorthCarolina_CameroonChapter_AngusIssues\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
d:\Angus_Issues\NorthCarolina_CameroonChapter_AngusIssues\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
d:\Angus_Issues\NorthCarolina_CameroonChapter_AngusIssues\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
d:\Angus_Issues\NorthCarolin